In [1]:
#  데이터 처리 모듈
import pandas as pd
# 행렬 등 통계 연산 모듈
import numpy as np
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

Mounted at /content/drive


### 1.한국신용데이터(KCD) 서울특별시 소상공인(SOHO)의 신용거래정보 추출 데이터를 활용한 파생변수 생성

* **credit_data_list**
  
  2022년 1월 ~ 2023년 3월 15개월의 서울특별시 소상공인 상권별 업종별 신용거래 및 사업장정보 데이터프레임을 담은 리스트 반환

* **변동계수(coefficient of variation, CV)**
  
  표준편차를 산술평균을 기준으로 표준화(standardization)한 것으로, 표준편차를 산술평균으로 나눈 값

  같은 단위를 가지는 표준편차를 평균으로 나누면, 단위가 사라지고 표준화된 수치를 비교

In [41]:
?class MergeData():

  def __init__(self, soho_credit_data = None):
    self.soho_credit_data = soho_credit_data

  def mergeSohoCredit(self, date_time1: int, date_time2: int,
                      file_path: str, file_name: str,
                      column_num:int, column_name_list: List):
    # date_time에 202201 ~ 202212, 202301 ~ 202303을 입력값으로 받고,
    # file_path에 해당 데이터가 있는 드라이브 경로 설정
    # file_path = /content/drive/MyDrive/2023BigContest/Archive/"

    credit_data_list: List = []
      # 월별 데이터프레임을 담을 리스트 생성
    date_time_list = [str(date_time1 + n) for n in range(0, 12, 1)]
    date_time_list.extend([str(date_time2 + n) for n in range(0, 3, 1)])

    for date in date_time_list:
      data = pd.read_csv(
          os.path.join(file_path + date + file_name),
          index_col = 0, sep = ",",
          names = column_name_list)
          # header = None)
      data.drop("index", axis = 0, inplace = True)
      data[column_name_list[column_num:]] = data[column_name_list[column_num:]].astype("float")

      # data["손익분기점_매출액_STD"] = data["월임대료"] / (1 - (data["부가가치세_STD"] + data["매입액"]) / V["매출액"]) = 고정비용 / (1-변동비율)
      # data["객단가_STD"] = data["매출액_STD"] / data["고객수_STD"]
      # data["신규고객단가_STD"] = data["매출액_STD"] / data["신규고객수_STD"]
      # data["전체대비_주말매출액비중_STD"] = data["주말매출액_STD"] / data["매출액_STD"]
        # 병합 후 파생컬럼 생성할 것

      credit_data_list.append(data)
        # os.path.join을 통한 구글 드라이브 파일 내 경로로 csv데이터 로드하여 판다스 데이터프레임으로 저장 후
        # credit_data_list에 0 ~ 11번째 인덱스의 원소로 추가

    return credit_data_list
      # 2022년 1월 ~ 2023년 3월 15개월의 서울특별시 소상공인 상권별 업종별 신용거래 및 사업장정보 데이터프레임을 담은 리스트 반환
      # 병합하여 self.soho_credit_data라는 멤버변수에 저장할 것

In [42]:
Preprocess = MergeData()

In [49]:
SOHO_INCOME_DATA_LIST = Preprocess.mergeSohoCredit(
    date_time1 = 202201,
    date_time2 = 202301,
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/SOHO_INCOME/SOHO_",
    file_name = "_INCOME_230916.csv",
    column_name_list = ["상권명", "상권코드", "외식업종_중분류",
                        "매출액_표준편차", "매입액_표준편차", "매출총이익_표준편차", "부가가치세_표준편차", "부가세차감전영업이익_표준편차", "영업이익_표준편차",
                        "주말매출액_표준편차", "사업장방문고객수_표준편차", "사업장방문신규고객수_표준편차",
                        "배달매출액_표준편차", "주말배달매출액_표준편차", "카드매출액_표준편차", "주말카드매출액_표준편차"],
    column_num = 3)

---

#### [SOHO_INCOME_DATA_LIST] 소호 신용거래정보 매출액(카드 + 배달), 매입액(카드 + 현금), 객단가, 부가가치세, 손익분기점 등 파생변수 생성

In [50]:
SOHO_INCOME_DATA_LIST[0].info()
SOHO_INCOME_DATA_LIST[14]

<class 'pandas.core.frame.DataFrame'>
Index: 170 entries, 0 to 169
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   상권명              170 non-null    object 
 1   상권코드             170 non-null    object 
 2   외식업종_중분류         170 non-null    object 
 3   매출액_표준편차         170 non-null    float64
 4   매입액_표준편차         170 non-null    float64
 5   매출총이익_표준편차       170 non-null    float64
 6   부가가치세_표준편차       170 non-null    float64
 7   부가세차감전영업이익_표준편차  170 non-null    float64
 8   영업이익_표준편차        170 non-null    float64
 9   주말매출액_표준편차       170 non-null    float64
 10  고객수_표준편차         170 non-null    float64
 11  신규고객수_표준편차       170 non-null    float64
 12  배달매출액_표준편차       154 non-null    float64
 13  주말배달매출액_표준편차     155 non-null    float64
 14  카드매출액_표준편차       170 non-null    float64
 15  주말카드매출액_표준편차     170 non-null    float64
dtypes: float64(13), object(3)
memory usage: 22.6+ KB


,상권명,상권코드,외식업종_중분류,매출액_표준편차,매입액_표준편차,매출총이익_표준편차,부가가치세_표준편차,부가세차감전영업이익_표준편차,영업이익_표준편차,주말매출액_표준편차,고객수_표준편차,신규고객수_표준편차,배달매출액_표준편차,주말배달매출액_표준편차,카드매출액_표준편차,주말카드매출액_표준편차
0,DMC(디지털미디어시티),2120098,카페,7.5727,5.2844,7.2851,4.7609,6.7029,10.8206,1.7354,18.2221,4.6328,0.7511,0.1341,7.6554,1.7541
1,가락시장역,2120234,한식,66.7188,3.3482,63.9273,25.6600,62.2638,84.1411,17.0500,6.9125,4.1122,15.3527,5.3017,64.8751,16.1978
2,가로수길,2120186,술집,24.6029,2.9186,24.4196,13.7904,21.0899,33.9835,11.4162,4.0024,3.7921,0.4714,0.1234,24.7073,11.4366
3,가로수길,2120186,양식,47.4696,5.5149,46.8361,26.5625,37.0288,62.5175,20.9340,14.4391,11.7155,0.3319,0.0989,47.5501,20.9460
4,가로수길,2120186,일식,43.2368,4.5679,45.0980,13.3462,43.9535,57.0685,7.4536,1.9332,1.7003,0.0008,0.0003,43.2368,7.4536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,홍대입구역(홍대),2120103,술집,49.1173,4.4199,47.4422,25.1515,45.5739,70.3352,25.2303,11.1215,7.0121,8.4986,3.0639,49.2780,25.3462
173,홍대입구역(홍대),2120103,양식,21.7228,4.6233,21.0248,15.6416,19.1330,34.4439,9.4146,5.9535,5.1722,6.3280,0.7660,21.9887,9.6435
174,홍대입구역(홍대),2120103,일식,14.7539,2.3404,14.2982,7.1473,16.0170,21.8626,6.4650,5.9388,4.7855,1.1049,0.3700,14.1164,6.2266
175,홍대입구역(홍대),2120103,카페,7.7766,2.5314,6.8361,3.6824,6.6363,7.0442,3.5475,8.6426,5.1141,3.3612,0.4447,7.2872,3.6716


```
* standard_deviation_population(kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery)
* standard_deviation_population(kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)
* standard_deviation_population((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash))
* standard_deviation_population(kcd.transactions.sum_sales_invoice - kcd.transactions.sum_purchase_invoice)
* standard_deviation_population(((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)) - kcd.transactions.monthly_rental_fee)
* standard_deviation_population((((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)) - kcd.transactions.monthly_rental_fee) - (kcd.transactions.sum_sales_invoice - kcd.transactions.sum_purchase_invoice))
* standard_deviation_population(kcd.transactions.sum_weekend_sales_card + kcd.transactions.sum_weekend_sales_delivery)
* standard_deviation_population(sum_customer_cnt)
* standard_deviation_population(sum_new_customer_cnt)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.sum_sales_delivery)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.sum_weekend_sales_delivery)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.sum_sales_card)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.sum_weekend_sales_card)
```


---

#### [SOHO_BUSINESS_DATA_LIST] 소호 임대동향, 사업장 정보

In [45]:
SOHO_BUSINESS_DATA_LIST = Preprocess.mergeSohoCredit(
    date_time1 = 202201,
    date_time2 = 202301,
    file_path = "/content/drive/MyDrive/2023BigConAVERAGEtest/data/DESILO_PLATFORM_DATA/SOHO_BUSINESS/SOHO_",
    file_name = "_BUSINESS_230911.csv",
    column_name_list = ["상권명", "외식업종_중분류",
                        "소호사업장_수", "경영위기사업장_총합", "프랜차이즈사업장_총합", "소상공인연령_평균", "소상공인업력_평균"],
    column_num = 2)

In [46]:
SOHO_BUSINESS_DATA_LIST[0].info()
SOHO_BUSINESS_DATA_LIST[14]

<class 'pandas.core.frame.DataFrame'>
Index: 172 entries, 0 to 171
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   상권명          172 non-null    object 
 1   외식업종_중분류     172 non-null    object 
 2   소호사업장_수      172 non-null    float64
 3   경영위기사업장_총합   172 non-null    float64
 4   프랜차이즈사업장_총합  172 non-null    float64
 5   소상공인연령_평균    172 non-null    float64
 6   소상공인업력_평균    172 non-null    float64
dtypes: float64(5), object(2)
memory usage: 10.8+ KB


,상권명,외식업종_중분류,소호사업장_수,경영위기사업장_총합,프랜차이즈사업장_총합,소상공인연령_평균,소상공인업력_평균
0,DMC(디지털미디어시티),카페,9.0000,4.0000,3.0000,40.1111,4.9556
1,가락시장역,한식,7.0000,0.0000,3.0000,43.1143,4.5429
2,가로수길,술집,6.0000,1.0000,2.0000,37.9167,5.0167
3,가로수길,양식,6.0000,2.0000,0.0000,45.7167,3.5667
4,가로수길,일식,6.0000,0.0000,1.0000,36.2333,3.7667
...,...,...,...,...,...,...,...
176,홍대입구역(홍대),술집,44.0000,4.0000,2.0000,39.4432,4.0000
177,홍대입구역(홍대),양식,10.0000,1.0000,2.0000,37.6500,5.8100
178,홍대입구역(홍대),일식,8.0000,1.0000,0.0000,34.9125,2.5375
179,홍대입구역(홍대),카페,16.0000,2.0000,0.0000,37.3000,4.6063


```
* COUNT(*)
* SUM(kcd.transactions.is_risky)
* SUM(kcd.meta-info.is_franchise)
* AVERAGE(kcd.meta-info.age)
* AVERAGE(kcd.meta-info.duration)
```

* ValueError: could not convert string to float: 'DMC(디지털미디어시티)'

---

#### [SOHO_METAINFO_DATA_LIST] 소호 개인사업자 업력, 연령, 경영위기 FLAG, 프랜차이즈(브랜드) TF


In [51]:
SOHO_METOINFO_DATA_LIST = Preprocess.mergeSohoCredit(
    date_time1 = 202201,
    date_time2 = 202301,
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/SOHO_METAINFO/SOHO_",
    file_name = "_METAINFO_230911.csv",
    column_name_list = ["상권명", "외식업종_중분류",
                        "정규고용인원_평균", "사업장방문고객수_평균", "사업장방문신규고객수_평균", "사업장임대면적_표준편차", "월임대료_표준편차", "임대보증금_표준편차"],
    column_num = 2)

In [52]:
SOHO_METOINFO_DATA_LIST[0].info()
SOHO_METOINFO_DATA_LIST[14]

<class 'pandas.core.frame.DataFrame'>
Index: 170 entries, 0 to 169
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   상권명            170 non-null    object 
 1   외식업종_중분류       170 non-null    object 
 2   정규고용인원_평균      170 non-null    float64
 3   사업장방문고객수_평균    170 non-null    float64
 4   사업장방문신규고객수_평균  170 non-null    float64
 5   사업장임대면적_표준편차   170 non-null    float64
 6   월임대료_표준편차      170 non-null    float64
 7   임대보증금_표준편차     170 non-null    float64
dtypes: float64(6), object(2)
memory usage: 12.0+ KB


,상권명,외식업종_중분류,정규고용인원_평균,사업장방문고객수_평균,사업장방문신규고객수_평균,사업장임대면적_표준편차,월임대료_표준편차,임대보증금_표준편차
0,DMC(디지털미디어시티),카페,0.7033,33.0211,8.2956,21.0944,0.9271,1.5947
1,가락시장역,한식,1.9414,7.8517,5.3167,43.2950,1.9838,2.9983
2,가로수길,술집,7.0683,4.4533,4.1133,51.4981,3.5908,3.4034
3,가로수길,양식,5.3750,12.0350,9.9800,232.0394,10.4152,16.8893
4,가로수길,일식,5.3750,3.3500,2.8883,63.1231,1.3525,3.0231
...,...,...,...,...,...,...,...,...
172,홍대입구역(홍대),술집,0.9261,9.2343,6.3218,66.8261,2.2330,3.4858
173,홍대입구역(홍대),양식,1.6330,8.1444,6.6267,82.3801,2.0107,3.9547
174,홍대입구역(홍대),일식,1.6675,6.7357,5.7886,56.7378,3.0323,2.5610
175,홍대입구역(홍대),카페,0.7244,12.5631,7.6494,78.9299,2.4548,4.3513


```
* AVERAGE(kcd.transactions.regular_employees_count)
* AVERAGE(kcd.transactions.sum_customer_cnt)
* AVERAGE(kcd.transactions.sum_new_customer_cnt)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.business_square_size)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.monthly_rental_fee)
* STANDARD_DEVIATION_POPULATION(kcd.transactions.rental_deposit)


---

#### [SOHO_STANDARDIZATION_DATA_LIST] 변동계수(coefficient of variation, CV) 산출을 위한 평균 연산

---

### 2.오아시스비즈니스(부동산 플랫폼 기업)의 서울특별시 고유필지단위별 소상공인의 매출등급 데이터에 상권코드, 법정동읍면동코드, 행정동코드 등 다른 테이블과의 병합(JOIN) 키로 활용될 컬럼 병합

* soho_sales_grade_list

  2022년 11개월의 서울특별시 고유필지단위별 소상공인의 매출등급 데이터를 담은 리스트 반환

* self.soho_commerce_data

  2022년 12개월의 소호 상권별 신용거래정보 데이터프레임을 담은 리스트와 2022년 11개월의 서울특별시 고유필지단위별 소상공인의 매출등급 데이터를 담은 리스트에서 원소(월별 데이터프레임)를 하나씩 꺼내어 병합하는 작업 11회 반복

In [ ]:
class LoadData():

  def __init__(self,
               commerce_district_data_path: str, commerce_district_data = None, soho_commerce_data = None):
    # 생성자를 활용하여 초기화하고 클래스 내부에 캡슐화

    commerce_district_data_chunk = pd.read_csv(
        commerce_district_data_path, chunksize = 10**5,
        index_col = False, na_values = "NaN", encoding = "utf-8")
    commerce_district_data_list = list(commerce_district_data_chunk)
      # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
    self.commerce_district_data = pd.concat(commerce_district_data_list)
    self.soho_commerce_data = soho_commerce_data


  def chunkDataMerge(self, column_name_list: List, codebook_filepath: str, district_file_path: str,
                     left_join_key1: str, right_join_key1: str, left_join_key2: str, right_join_key2: str):
    self.commerce_district_data.columns = column_name_list
    codebook = pd.read_csv(codebook_filepath, index_col = False)
    district_code = pd.read_csv(district_file_path, index_col = False, encoding = "euc-kr")

    self.commerce_district_data = pd.merge(
        left = self.commerce_district_data, right = codebook,
        left_on = left_join_key1, right_on = right_join_key1, how = "left")
    # display(self.commerce_district_data.info())
    # display(district_code.info())

    self.commerce_district_data.drop(right_join_key1, axis = 1, inplace = True)
    self.commerce_district_data = pd.merge(
        left = self.commerce_district_data, right = district_code[["상권_구분_코드_명", "상권_코드", "상권_코드_명"]],
        left_on = left_join_key2, right_on = right_join_key2, how = "left")

    return self.commerce_district_data


  def splitMonthlyData(self, month: int):
    soho_sales_grade_list: List = []
    while month < 202213:
      soho_sales_grade_list.append(
          self.commerce_district_data.loc[self.commerce_district_data["데이터기준연월"] == month])
      month += 1

    return soho_sales_grade_list


  def mergeData(self, soho_credit_list: List, sales_grade_list: List,
                left_join_key: str, right_join_key: str):
    self.soho_commerce_data: List = []
    for index in range(0, 11, 1):
      self.soho_commerce_data(
          pd.merge(left = soho_credit_list[index], right = sales_grade_list[index],
                  left_on = left_join_key, right_on = right_join_key, how = "left"))

    return self.soho_commerce_data

In [ ]:
soho_sales_grade = LoadData(
    commerce_district_data_path = "/content/drive/MyDrive/2023BigContest/data/대회제공/필지단위 소상공인 매출등급 정보.csv")
commerce_district_sales = soho_sales_grade.chunkDataMerge(
    column_name_list = ["데이터기준연월", "필지고유번호", "법정동읍면동코드", "업종코드", "매출등급"],
    codebook_filepath = "/content/drive/MyDrive/2023BigContest/data/market_loc_map.csv",
    left_join_key1 = "법정동읍면동코드", right_join_key1 = "STDG_EMD_CD",
    district_file_path = "/content/drive/MyDrive/2023BigContest/data/서울시 상권분석서비스(상권영역).csv",
    left_join_key2 = "상권_코드", right_join_key2 = "상권_코드")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13138513 entries, 0 to 13138512
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   데이터기준연월      int64  
 1   필지고유번호       float64
 2   법정동읍면동코드     int64  
 3   업종코드         object 
 4   매출등급         int64  
 5   상권_코드        float64
 6   행정동_코드       float64
 7   STDG_EMD_CD  float64
dtypes: float64(4), int64(3), object(1)
memory usage: 902.2+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650 entries, 0 to 1649
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   기준_년월_코드    1650 non-null   int64  
 1   상권_구분_코드    1650 non-null   object 
 2   상권_구분_코드_명  1650 non-null   object 
 3   상권_코드       1650 non-null   int64  
 4   상권_코드_명     1650 non-null   object 
 5   엑스좌표_값      1650 non-null   int64  
 6   와이좌표_값      1650 non-null   int64  
 7   시군구_코드      1650 non-null   int64  
 8   행정동_코드      1650 non-null   int64  
 9   형태정보        0 non-null      float64
dtypes: float64(1), int64(6), object(3)
memory usage: 129.0+ KB


None

In [ ]:
soho_sales_grade_list = soho_sales_grade.splitMonthlyData(month = 202201)

```
STDG_EMD_CD_codebook = pd.read_excel(
    "/content/drive/MyDrive/2023BigContest/data/대회제공/소상공인 매출등급 예상 대상 필지.xlsx",
    sheet_name = 0,
    index_col = None)
STDG_EMD_CD_codebook.drop(["SLS_GRD_2301", "SLS_GRD_2302"], axis = 1, inplace = True)
  # [법정동읍면동코드] 시, 구 아래에 두며 그 아래에는 통과 반이 있는, 법률로 지정한 행정 구역의 정보를 나타내는 기호 체계
  # 법정동(10자리)의 구성: 광역시(2자리)+시군구(3자리)+읍면동(3자리)+리(2자리)
  # 법정동코드 10자리 중 앞6번째부터 8번째자리(101:청운동, 101:신교동…)
display(STDG_EMD_CD_codebook)
  # 고유식별자는 총 10,000행
  # [매출등급] 특정 필지의 '평당추정매출/평당월임대료'
```

In [ ]:
display(credit_data_list[10]), display(soho_sales_grade_list[10])
  # 2022년과 2023의 상권코드는 체계가 다른것으로 추정되어 TABLE JOIN하였으나 모두 NULL값으로 반환

,상권명,매출액_STD,매입액_STD,매출총이익_STD,부가가치세_STD,부가세차감전영업이익_STD,영업이익_STD,주말매출액_STD,고객수_STD,신규고객수_STD,객단가_STD,신규고객단가_STD,전체대비_주말매출액비중_STD
0,DMC(디지털미디어시티),8.7375,3.3341,6.9208,7.8986,6.3991,13.0150,1.4755,21.0679,4.3491,0.4147,2.0090,0.1689
1,가락시장역,70.3739,3.2990,67.1710,24.5759,65.5177,89.8393,20.2020,6.9940,4.3547,10.0621,16.1606,0.2871
2,가로수길,39.7228,4.6042,38.5268,18.1484,35.6419,52.7261,14.5716,8.8283,6.9489,4.4995,5.7164,0.3668
3,가산디지털단지,18.5851,3.9317,17.4122,9.8264,16.3233,24.9982,2.3073,13.4041,3.9728,1.3865,4.6781,0.1241
4,강남구청역,21.5684,2.9764,21.3244,12.5425,21.0141,33.0387,5.5877,10.0100,4.6761,2.1547,4.6125,0.2591
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,학동사거리,22.4856,6.4159,20.8802,26.6258,19.5931,38.8400,9.8561,4.0100,2.3598,5.6074,9.5286,0.4383
89,합정역,23.0361,2.5274,22.7046,17.8011,22.2582,34.8829,8.7438,4.5823,3.7301,5.0272,6.1758,0.3796
90,해방촌 남동측,26.2127,2.2124,25.1345,7.2755,24.9448,31.6445,11.5609,4.4983,3.2644,5.8273,8.0299,0.4410
91,홍대입구역 3번,2.6275,2.9233,4.2541,2.0431,4.2693,4.5203,1.4709,0.7741,0.7896,3.3941,3.3277,0.5598


,데이터기준연월,필지고유번호,법정동읍면동코드,업종코드,매출등급,상권_코드,행정동_코드,상권_구분_코드_명,상권_코드_명
12001606,202211,1111010000000000000.0000,11110101,A03,4,2110008.0000,11110515.0000,NaN,NaN
12001607,202211,1111010000000000000.0000,11110101,A03,4,2110011.0000,11110515.0000,NaN,NaN
12001608,202211,1111010000000000000.0000,11110101,A03,4,2110014.0000,11110515.0000,NaN,NaN
12001609,202211,1111010000000000000.0000,11110101,A03,4,2110015.0000,11110515.0000,NaN,NaN
12001610,202211,1111010000000000000.0000,11110101,A03,4,2130001.0000,11110515.0000,NaN,NaN
...,...,...,...,...,...,...,...,...,...
13138508,202211,1168010000000000000.0000,11680108,A03,3,2110955.0000,11680521.0000,NaN,NaN
13138509,202211,1168010000000000000.0000,11680108,A03,3,2120185.0000,11680521.0000,NaN,NaN
13138510,202211,1168010000000000000.0000,11680108,A03,3,2120187.0000,11680521.0000,NaN,NaN
13138511,202211,1168010000000000000.0000,11680108,A03,3,2120191.0000,11680521.0000,NaN,NaN


(None, None)

In [ ]:
soho_commerce_merge_data = soho_sales_grade.mergeData(
    soho_credit_list = credit_data_list, sales_grade_list = soho_sales_grade_list,
    left_join_key = "상권코드", right_join_key = "상권_코드")

```
# VarianceInflationFactor()
# VIF(분산팽창인자) 값에 따른 최소한 모형 적합 시 입력할 독립변수의 수 파악을 위한 반복문 수행
  def VarainceInflationFactor(self,
                              X_train_data: pd.DataFrame(), X_test_data: pd.DataFrame(),
                              # num_variables: int,
                              Y_train_data: pd.DataFrame(), Y_test_data: pd.DataFrame()):

    evaluation_train = pd.DataFrame()
    evaluation_test = pd.DataFrame()

    for index in tqdm( range(1, len(X_train_data.columns) + 1)):

    # 분산팽창인자를 피처 수별로 구한 후 오름차순하여 vif 데이터프레임에 저장(다중공선성 문제로 나머지 변수 제외)
    # X_train_data에서 변수 index 수만큼의 피처(독립변수)만 남기고 나머지 변수 제외하여 X_train_fe_scaling_multiple 생성
    # X_train_test도 이하 상동
      vif = pd.DataFrame()
      vif["VIF_factor"] = [variance_inflation_factor(X_train_data.values, index)
                          for index in range(X_train_data.shape[1]) ]
      vif["Features"] = X_train_data.columns
      # print(vif)
      X_column_vif = vif.sort_values(by = "VIF_factor", ascending = True)["Features"][:index].values
        # TypeError: 'NoneType' object is not subscriptable

      X_train_fe_scaling_multiple, X_test_fe_scaling_multiple = X_train_data[X_column_vif].copy(), X_test_data[X_column_vif].copy()
```